In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

def simulate_ball_on_plate(delta_t=0.1, friction=0.98, tsx=0.0, tsy=0.0, Kp=4.0, Ki=2.0, Kd=1.0):
    """
    Simulate a ball on a plate using a PID controller to control the position of the ball.

    Parameters:
    - friction: Friction coefficient of the plate.
    - tsx: Target position in x direction.
    - tsy: Target position in y direction.
    - Kp: Proportional gain for PID controller.
    - Ki: Integral gain for PID controller.
    - Kd: Derivative gain for PID controller.

    The simulation calculates the position and angles of the ball on the plate over a number of steps,
    applying the PID controller to adjust the roll and pitch angles based on the target positions.
    """
    # Initial conditions
    roll = 0.0 # Current roll angle in degrees
    pitch = 0.0 # Current pitch angle in degrees
    sx = 0.2 # Current Position in meters
    sy = 0.2 # Current Position in meters
    vx = 0.0 # Current velocity in m/s
    vy = 0.0 # Current velocity in m/s
    g = 9.81 # Gravity g in m/s^2
    # delta_t = 0.1 # Interval in seconds (s)
    max_angle = np.radians(4) # Max angels
    steps = 100 # Steps

    # Additional varibales
    max_agular_speed = 20.0 # [degree] (degree per second)
    servo_noise_std = 0.1

    # State variables for PID controller
    # Integral and previous error terms for PID control
    integral_x = 0.0
    integral_y = 0.0
    prev_error_x = 0.0
    prev_error_y = 0.0

    # Storage for positions and angles
    positions_x = []
    positions_y = []
    angles_x = []
    angles_y = []
    delta_x = []
    delta_y = []
    
    # Iterate over the number of steps
    for _ in range(steps):
        # Calculate accelerate for x
        roll_theta = roll
        ax = (3/5) * g * np.sin(roll_theta)

        # Calculate accelerate for y
        pitch_theta = pitch
        ay = (3/5) * g * np.sin(pitch_theta)

        # Calculcate velocity for x
        vx = vx + ax * delta_t

        # Calculcate velocity for y
        vy = vy + ay * delta_t

        # friction
        vx *=friction
        vy *=friction

        # Calculcate position for x
        sx = sx + vx * delta_t + (1/2) * ax * (delta_t * delta_t)

        # Calculcate position for y
        sy = sy + vy * delta_t + (1/2) * ay * (delta_t * delta_t)

        # Degree times delta time
        max_delta_angle = max_agular_speed * delta_t

        # PID for x
        ex = tsx - sx
        integral_x += ex * delta_t
        derivative_x = (ex - prev_error_x) / delta_t
        prev_error_x = ex
        ux = Kp * ex + Ki * integral_x + Kd * derivative_x

        # Set roll
        desire_roll = np.clip(ux, -max_angle, max_angle)
        delta_roll = desire_roll - roll
        roll += np.clip(delta_roll, -max_delta_angle, max_delta_angle)

        positions_x.append(sx)
        angles_x.append(np.degrees(desire_roll))
        delta_x.append(np.clip(delta_roll, -max_delta_angle, max_delta_angle))

        # PID for y
        ey = tsy - sy
        integral_y += ey * delta_t
        derivative_y = (ey - prev_error_y) / delta_t
        prev_error_y = ey
        uy = Kp * ey + Ki * integral_y + Kd * derivative_y

        # # Set pitch
        # Set roll
        desire_pitch = np.clip(uy, -max_angle, max_angle)
        delta_pitch = desire_pitch - pitch
        pitch += np.clip(delta_pitch, -max_delta_angle, max_delta_angle)

        positions_y.append(sy)
        angles_y.append(np.degrees(pitch))
        delta_y.append(np.clip(delta_pitch, -max_delta_angle, max_delta_angle))

    # Plot
    fig, ax = plt.subplots(3, 2, figsize=(10, 8))

    ax[0, 0].plot(positions_x)
    ax[0, 0].set(title='Position x',  ylabel='m')

    ax[1, 0].plot(angles_x)
    ax[1, 0].set(title='Rollwinkel',  ylabel='°')

    ax[2, 0].plot(delta_x)
    ax[2, 0].set(title='Delta x',  ylabel='m', xlabel='Zeitschritt')
    
    ax[0, 1].plot(positions_y)
    ax[0, 1].set(title='Position y',  ylabel='m', xlabel='Zeitschritt')
    
    ax[1, 1].plot(angles_y)
    ax[1, 1].set(title='Pitchwinkel', ylabel='°', xlabel='Zeitschritt')

    ax[2, 1].plot(delta_y)
    ax[2, 1].set(title='Delta y',  ylabel='m', xlabel='Zeitschritt')

    plt.show()

# Interactive widget to control the simulation parameters
interact(simulate_ball_on_plate,
         delta_t=FloatSlider(min=0.01, max=0.2, step=0.01, value=0.1, description='detla_t (s)'),
         friction=FloatSlider(min=0, max=1, step=0.005, value=0.98, description='Friction'),
         tsx=FloatSlider(min=-1, max=1, step=0.05, value=0.0, description='Target X'),
         tsy=FloatSlider(min=-1, max=1, step=0.05, value=0.0, description='Target Y'),
         Kp=FloatSlider(min=0, max=10, step=0.1, value=1.0, description='Kp'),
         Ki=FloatSlider(min=0, max=10, step=0.1, value=0.0, description='Ki'),
         Kd=FloatSlider(min=0, max=10, step=0.1, value=0.5, description='Kd'))